[Back to the starting page](start.ipynb)

In [1]:
import numpy as np
from scripts import *

In [2]:
# 1-D isostatic comparison of reference columns
#
# 1) definition of the reference 125 km thick continental lithosphere
# 2) definition of the MOR column (model M2)
#
# Remark:
# sea level and dh (elevation difference between continent and MOR)
# are given relative to the top of the continental column, i.e. negative = down
#
water_depth_at_ridge     = 2750.0e0  # meters = reference water depth at the ridge based on average MOR elevation
sediment_thickness_fixed = 0.0e0     # meters
sea_level_ref            = -400e0    # meters = reference sea level based on average elevation of continents
dz                       = 200e0     # meters

# Lithosphere 1 = Reference continental column
reflith                  = 'lith125'
book_reflith             = book_lith125_GL # updated parameters (./scripts/book_lith.py)

# Lithosphere 2 = Compared column
# The MOR from 2-D thermechanical model is used to calibrate the elevation of the reference continental column
# When you compare two continental column, it should be no elevation difference
#
complith                 = 'MOR'  # 'MOR' 'lith160' 'lith180' 'lith200' 'lith240' 'lith280'

use_perplex_table_mantle = False
perplex_name             = 'slm_sp2008'   # composition of the continental lithospheric mantle

# Select right set of parameters, updated parameters (./scripts/book_lith.py)
#
if complith == 'MOR':
    book_complith  = book_ridge
    ref_depth      = 125e3    # meters = compensation depth (base of the contiental lithosphere)
elif complith == 'lith160':
    book_complith  = book_lith160_GL
    ref_depth      = 160e3    # meters = compensation depth (base of the contiental lithosphere)        
elif complith == 'lith180':
    book_complith  = book_lith180_GL
    ref_depth      = 180e3    # meters = compensation depth (base of the contiental lithosphere)
elif complith == 'lith200':
    book_complith  = book_lith200_GL
    ref_depth      = 200e3    # meters = compensation depth (base of the contiental lithosphere)
elif complith == 'lith240':
    book_complith  = book_lith240_GL
    ref_depth      = 240e3    # meters = compensation depth (base of the contiental lithosphere)
elif complith == 'lith280':
    book_complith  = book_lith280_GL
    ref_depth      = 280e3    # meters = compensation depth (base of the contiental lithosphere)

if use_perplex_table_mantle:
    # (see ./scripts/scripts_geodyn1d.py)
    # - turn on reading perplex grid option
    # - define the correct perplex grid
    # - use the right degree of depletion in the continental lithospheric mantle to fit MOR/continets elevation
    #
    book_reflith  = update_mantle_density(reflith, book_reflith, perplex_name=perplex_name)  
    book_complith = update_mantle_density(complith,book_complith,perplex_name=perplex_name)
else:
    book_reflith  = update_mantle_density(reflith, book_reflith, perplex_name=None)  
    book_complith = update_mantle_density(complith,book_complith,perplex_name=None)

    
print("# Column 1: Reference Continental lithosphere ###############################################################")
lith1 = geodyn1d.lithosphere(reflith,printOut=False)
lith1.update_materials(book_reflith)
lith1.get_steady_state_geotherm(dz=dz,printOut=True)
lith1.get_pressure_density_profile(printOut=False,path='./data/thermodyn/'+perplex_name)
pref  = lith1.get_reference_pressure(ref_depth,printOut=True)
print('Pref = {0:7.5e} Pa with compensation depth of {1:5.1f} km'.format(pref,ref_depth/1e3))
# compute average crustal density for information
rhoc = lith1.P_rho_DepthProfile.rho[lith1.P_rho_DepthProfile.Z<35000]
depc = lith1.P_rho_DepthProfile.Z[lith1.P_rho_DepthProfile.Z<35000]
print("====> Average crustal density "+reflith+" is {} kg/m3".format(np.mean(rhoc)))

print("\n# Column 2: "+complith+" lithosphere ########################################################################")
if complith == 'MOR':
    if use_perplex_table_mantle:
        deltarho_ridge_correction        = -0.2 # kg/m3 # correction to reproduce the elevation difference 
                                                # in the 2-D thermo-mechanical model
        geotherm_filename                = 'profile_temperature_ridge_centralvalley_GL-Tp1280_speed3-perplex_corrected.tz'
        density_profile_filename_ridge   = 'profile_density_ridge_centralvalley_GL-Tp1280_speed3-perplex.rhoz'
        path                             = './data/geodyn2d/M1/'
    else:
        deltarho_ridge_correction        = 4.5  # kg/m3 # correction to reproduce the elevation difference 
                                                # in the 2-D thermo-mechanical model
        geotherm_filename                = 'profile_temperature_ridge_centralvalley_GL-Tp1280_speed3_corrected.tz'
        density_profile_filename_ridge   = 'profile_density_ridge_centralvalley_GL-Tp1280_speed3.rhoz'
        path                             = './data/geodyn2d/M2/'

    water_depth_used = water_depth_at_ridge
    lith2            = geodyn1d.lithosphere('ridge_Oc6_5_DiffLayer26',printOut=False)
    lith2.update_materials(book_complith)
    lith2.get_imposed_geotherm(path+geotherm_filename,printOut=True)
    lith2.get_pressure_density_profile(printOut=False,
                                       filename=path+density_profile_filename_ridge,
                                       drho=deltarho_ridge_correction,
                                       zdrho=[6500,125000])
else:
    water_depth_used = 0
    lith2            = geodyn1d.lithosphere(complith,printOut=False)
    lith2.update_materials(book_complith)
    lith2.get_steady_state_geotherm(printOut=True)
    lith2.get_pressure_density_profile(printOut=False,path='./data/thermodyn/'+perplex_name)
    # compute average crustal density for information
    rhoc = lith2.P_rho_DepthProfile.rho[lith2.P_rho_DepthProfile.Z<35000]
    depc = lith2.P_rho_DepthProfile.Z[lith2.P_rho_DepthProfile.Z<35000]
    print("====> Average crustal density "+complith+" is {} kg/m3".format(np.mean(rhoc)))
    
print("\n# Subsidence at the "+complith+" ###############################################################")
#-------------------------------------------------------------------
lith2.set_sediment_thickness(sediment_thickness_fixed)
lith2.set_water_depth(water_depth_used)

dh,water_depth,sea_level,sediment_thickness = lith2.get_subsidence_with_sediments(pref,
                                                                                  ref_depth,
                                                                                  SeaLevelFixed=False,
                                                                                  printOut=False)

print('========= SUBSIDENCE AT THE RIDGE WITH FIXED WATER DEPTH ===========')
print('WATER DEPTH        = {}'.format(water_depth))
print('SEA LEVEL          = {}'.format(sea_level))
print('SEDIMENT THICKNESS = {}'.format(sediment_thickness))
print('dh                 = {}'.format(dh))
# Stored for the plot in the next cell
dh_fixed_water_depth = dh
water_depth_final    = water_depth

#-------------------------------------------------------------------
lith2.set_sediment_thickness(sediment_thickness_fixed)
lith2.set_sea_level(sea_level_ref)   #-1.e-9)

dh,water_depth,sea_level,sediment_thickness = lith2.get_subsidence_with_sediments(pref,
                                                                                  ref_depth,
                                                                                  SeaLevelFixed=True,
                                                                                  printOut=False)
    

print('========= SUBSIDENCE AT THE RIDGE WITH FIXED SEA LEVEL ===========')
print('WATER DEPTH        = {} m'.format(water_depth))
print('SEA LEVEL          = {} m'.format(sea_level))
print('SEDIMENT THICKNESS = {} m'.format(sediment_thickness))
print('dh                 = {} m'.format(dh))

#-------------------------------------------------------------------
lith2.set_sediment_thickness(sediment_thickness_fixed)
lith2.set_sea_level(-10000.)

dh,water_depth,sea_level,sediment_thickness = lith2.get_subsidence_with_sediments(pref,
                                                                                  ref_depth,
                                                                                  SeaLevelFixed=True,
                                                                                  printOut=False)

print('========= SUBSIDENCE AT THE RIDGE WITH FIXED SEA LEVEL ===========')
print('WATER DEPTH        = {} m'.format(water_depth))
print('SEA LEVEL          = {} m (=> no water load)'.format(sea_level))
print('SEDIMENT THICKNESS = {} m'.format(sediment_thickness))
print('dh                 = {} m'.format(dh))


# Column 1: Reference Continental lithosphere ###############################################################
 z bottom (km) = 600000.0
 k asth. = 87.56
 Grad slm = 0.4
 Qbasal = 0.0195488281
 Tbasal = 1520.0
 Tp = 1280.0
Layer 6 T top = 1330.0 Q top = 19.55 (matSLM)
Layer 3 T top =  548.0 Q top = 19.55 (matLM1)
Layer 2 T top =  441.8 Q top = 28.61 (matLC)
Layer 1 T top =  295.3 Q top = 37.68 (matMC)
Layer 0 T top =    0.0 Q top = 51.28 (matUC)
 Qsurface = 51.28 mW/m2
Pref = 3.79540e+09 Pa with compensation depth of 125.0 km
====> Average crustal density lith125 is 2833.0813751716532 kg/m3

# Column 2: MOR lithosphere ########################################################################

# Subsidence at the MOR ###############################################################
========= SUBSIDENCE AT THE RIDGE WITH FIXED WATER DEPTH ===========
WATER DEPTH        = 2750.0
SEA LEVEL          = -412.0041304694064
SEDIMENT THICKNESS = 1e-09
dh                 = -3162.0041304704064
=======

In [6]:
# Display temperature, pressure, and density profiles of the two compared columns
#
# For lithosphere 2:
#     Pressure is shifted with water load
#     Depth is shifted with relative elevation between lith1 and lith2
#

%matplotlib widget

define_rcParams()

fig=plt.figure(figsize=[10/2.54,10/2.54])
fig.subplots_adjust(hspace=0.4, wspace=0.4)
plt.plot(lith2.geotherm.T-273.15,
         lith2.geotherm.Z/1e3-dh_fixed_water_depth/1e3,
         label="Lithosphere 2")
plt.plot(lith1.geotherm.T-273.15,lith1.geotherm.Z/1e3,label="Ref. 125 km cont. lith.")
plt.xlim([0,1600])
plt.ylim([0,ref_depth/1e3+5])
plt.xlabel(r'Temperature ($^{\circ}C$)')
plt.ylabel('Depth (km)')
plt.legend()
plt.gca().invert_yaxis()

fig=plt.figure(figsize=[10/2.54,10/2.54])
plt.plot(lith2.P_rho_DepthProfile.P/1e6+water_depth_final*10*1030/1e6,
         lith2.P_rho_DepthProfile.Z/1e3-dh_fixed_water_depth/1e3,
         label="Lithosphere 2")
plt.plot(lith1.P_rho_DepthProfile.P/1e6,lith1.P_rho_DepthProfile.Z/1e3,label="Ref. 125 km cont. lith.")
plt.ylim([0,ref_depth/1e3+5])
plt.xlim([0,ref_depth*10*np.mean(lith1.P_rho_DepthProfile.rho)/1e6+200])
plt.xlabel('Pressure (MPa)')
plt.ylabel('Depth (km)')
plt.legend()
plt.gca().invert_yaxis()

fig=plt.figure(figsize=[10/2.54,10/2.54])
plt.plot(lith2.P_rho_DepthProfile.rho,
         lith2.P_rho_DepthProfile.Z/1e3-dh_fixed_water_depth/1e3,
         label="Lithosphere 2")
plt.plot(lith1.P_rho_DepthProfile.rho,
         lith1.P_rho_DepthProfile.Z/1e3,
         label="Ref. 125 km cont. lith.")
plt.ylim([0,ref_depth/1e3+5])
plt.xlim([2700,3600])
plt.xlabel(r'Density ($kg/m^{3}$)')
plt.ylabel('Depth (km)')
plt.legend()
plt.gca().invert_yaxis()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …